In [1]:
import sys
import os
sys.path.append("/Users/satyaprk/Documents/intern_codes/robustness_vs_counterfactuals/")
sys.path.append("/Users/skrishna/Documents/phd_codes/neurips_paper/")
sys.path.append("/Users/satyaprk/Documents/intern_codes/robustness_vs_counterfactuals/Recourse_Methods/AR")
sys.path.append("/Users/satyaprk/Documents/intern_codes/robustness_vs_counterfactuals/Recourse_Methods/Generative_Model")
sys.path.append("/Users/satyaprk/Documents/intern_codes/robustness_vs_counterfactuals/Recourse_Methods/AR/recourse/")



In [2]:

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import torch.nn as nn
import pickle as pkl
from numpy import linalg as LA
import torch.nn as nn

import ML_Models.data_loader as loader
# from utils import _get_input_subset
# from Recourse_Methods.gradient_methods import SCFE
# from utils import get_recourses, get_performance_measures


%matplotlib inline
%config InlineBackend.figure_format = 'svg'



In [3]:
## Dataset Prep


from torchvision import  datasets, transforms
from torch.utils.data import DataLoader


data_name = "adult"

n_starting_instances = 1200
compas_dict = {
        "data_path": '../Data_Sets/COMPAS/',
        "filename_train": 'compas-train-upsampled.csv',
        "filename_test": 'compas-test-upsampled.csv',
        "label": "risk",
        "task": "classification",
        "lr": 1e-3,
        "d": 6,
        "H1": 10,
        "H2": 10,
        "activFun": nn.Softplus(),
        "n_starting_instances": n_starting_instances
    }

german_dict = {
        "data_path": '../Data_Sets/German_Credit_Data/',
        "filename_train": 'german-train.csv',
        "filename_test": 'german-test.csv',
        "label": "credit-risk",
        "task": "classification",
        "lr": 1e-3,
        "d": 6,
        "H1": 10,
        "H2": 10,
        "activFun": nn.Softplus(),
        "n_starting_instances": n_starting_instances
    
    }

adult_dict = {
        "data_path": "../Data_Sets/Adult/",
        "filename_train": 'adult-train.csv',
        "filename_test": 'adult-test.csv',
        "label": 'income',
        "task": "classification",
        "lr": 1e-3,
        "d": 6,
        "H1": 25,
        "H2": 25,
        "activFun": nn.Softplus(),
        "n_starting_instances": n_starting_instances
    }


data_meta_dictionaries = {
        "compas": compas_dict, 
        "adult": adult_dict, 
        "german":german_dict
    }
data_meta_info = data_meta_dictionaries[data_name]




In [4]:
train_data = pd.read_csv(os.path.join(data_meta_info["data_path"], data_meta_info["filename_train"]))
test_data = pd.read_csv(os.path.join(data_meta_info["data_path"], data_meta_info["filename_test"]))




In [5]:
train_data[data_meta_info["label"]] = train_data[data_meta_info["label"]].replace(0,-1)

test_data[data_meta_info["label"]] = test_data[data_meta_info["label"]].replace(0,-1)


In [6]:
data_meta_info["data_path"]

'../Data_Sets/Adult/'

In [7]:
data_meta_info["filename_train"]

'adult-train.csv'

In [8]:
train_data

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,sex_Male,workclass_Private,marital-status_Non-Married,occupation_Other,relationship_Non-Husband,race_White,native-country_US,income
0,49,269417,16,0,2258,50,0,0,1,0,1,1,1,1
1,26,186151,9,0,0,40,0,1,1,1,1,1,1,-1
2,38,33394,9,0,0,75,1,1,0,1,0,1,1,1
3,33,192644,16,0,1977,90,1,1,0,0,0,1,1,1
4,50,139296,7,0,0,40,1,0,1,1,1,1,1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36172,27,133770,13,2202,0,52,1,1,1,0,1,0,0,-1
36173,36,111268,12,0,0,40,0,1,1,1,1,1,1,-1
36174,25,345121,6,0,0,25,0,1,1,1,1,1,1,-1
36175,51,145409,13,15024,0,50,1,0,0,1,0,1,1,1


In [10]:
train_data.to_csv(os.path.join(data_meta_info["data_path"], "{}-train-label.csv".format(data_name)), index = False)
test_data.to_csv(os.path.join(data_meta_info["data_path"], "{}-test-label.csv".format(data_name)), index = False)